In [100]:
%pip install scikit-hts[prophet]

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in c:\users\andre\appdata\local\programs\python\python38\lib\site-packages (from matplotlib>=2.0.0->fbprophet==0.7.1->scikit-hts[prophet]) (2.4.6)


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\andre\appdata\local\programs\python\python38\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\andre\\AppData\\Local\\Temp\\pip-install-x75c7aly\\fbprophet_22313a496890464e866ed6b519f7f8ef\\setup.py'"'"'; __file__='"'"'C:\\Users\\andre\\AppData\\Local\\Temp\\pip-install-x75c7aly\\fbprophet_22313a496890464e866ed6b519f7f8ef\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\andre\AppData\Local\Temp\pip-wheel-ij8b1la3'
       cwd: C:\Users\andre\AppData\Local\Temp\pip-install-x75c7aly\fbprophet_22313a496890464e866ed6b519f7f8ef\
  Complete output (43 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  cre


  Running setup.py clean for fbprophet
Failed to build fbprophet
    Running setup.py install for fbprophet: started
    Running setup.py install for fbprophet: finished with status 'error'


    File "C:\Users\andre\AppData\Local\Temp\pip-install-x75c7aly\fbprophet_22313a496890464e866ed6b519f7f8ef\setup.py", line 38, in build_models
      StanBackendEnum.get_backend_class(backend).build_model(target_dir, MODEL_DIR)
    File "C:\Users\andre\AppData\Local\Temp\pip-install-x75c7aly\fbprophet_22313a496890464e866ed6b519f7f8ef\fbprophet\models.py", line 209, in build_model
      import pystan
    File "c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pystan\__init__.py", line 9, in <module>
      from pystan.api import stanc, stan
    File "c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pystan\api.py", line 13, in <module>
      import pystan._api  # stanc wrapper
  ImportError: DLL load failed while importing _api: The specified module could not be found.
  ----------------------------------------
  ERROR: Failed building wheel for fbprophet
    ERROR: Command errored out with exit status 1:
     command: 'c:\users\andre\appdata\l

In [101]:
%pip install fbprophet

  Using cached fbprophet-0.7.1.tar.gz (64 kB)


  ERROR: Command errored out with exit status 1:
   command: 'c:\users\andre\appdata\local\programs\python\python38\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\andre\\AppData\\Local\\Temp\\pip-install-94530qvc\\fbprophet_1085d65114d44eef987c19de93aa7d19\\setup.py'"'"'; __file__='"'"'C:\\Users\\andre\\AppData\\Local\\Temp\\pip-install-94530qvc\\fbprophet_1085d65114d44eef987c19de93aa7d19\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\andre\AppData\Local\Temp\pip-wheel-4eck8id8'
       cwd: C:\Users\andre\AppData\Local\Temp\pip-install-94530qvc\fbprophet_1085d65114d44eef987c19de93aa7d19\
  Complete output (43 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  cre


  Running setup.py clean for fbprophet
Failed to build fbprophet
    Running setup.py install for fbprophet: started
    Running setup.py install for fbprophet: finished with status 'error'


    File "c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\wheel\bdist_wheel.py", line 223, in run
      self.run_command('build')
    File "c:\users\andre\appdata\local\programs\python\python38\lib\distutils\cmd.py", line 313, in run_command
      self.distribution.run_command(command)
    File "c:\users\andre\appdata\local\programs\python\python38\lib\distutils\dist.py", line 985, in run_command
      cmd_obj.run()
    File "c:\users\andre\appdata\local\programs\python\python38\lib\distutils\command\build.py", line 135, in run
      self.run_command(cmd_name)
    File "c:\users\andre\appdata\local\programs\python\python38\lib\distutils\cmd.py", line 313, in run_command
      self.distribution.run_command(command)
    File "c:\users\andre\appdata\local\programs\python\python38\lib\distutils\dist.py", line 985, in run_command
      cmd_obj.run()
    File "C:\Users\andre\AppData\Local\Temp\pip-install-94530qvc\fbprophet_1085d65114d44eef987c19de93aa7d19\setup.py", 

In [50]:
from datetime import datetime
from hts import HTSRegressor
from hts.utilities.load_data import load_hierarchical_sine_data
import hts.functions
import calcbench as cb

cb.api_client._rig_for_testing("localhost")

In [61]:
d = cb.standardized_data(
    company_identifiers=["msft", "orcl", "ko"],
    metrics=["GrossProfit", "SIC_Code"],
    all_history=True,
    period_type="annual",
)

In [62]:
one_digit_SIC_codes = pd.to_numeric(
    d["SIC_Code"].iloc[0] // 1000, downcast="integer"
).astype(str)
one_digit_SIC_codes.name = "SIC_Code"

In [63]:
d = d.drop(columns="SIC_Code", level="metric")

In [64]:
d = d.drop(d.index[0]) # drop the 0000 column left over from the SIC_Code

In [65]:
d = d.stack().join(one_digit_SIC_codes).set_index("SIC_Code", append=True)

In [78]:
d.index.levels[0].to_timestamp()

DatetimeIndex(['2007-01-01', '2008-01-01', '2009-01-01', '2010-01-01',
               '2011-01-01', '2012-01-01', '2013-01-01', '2014-01-01',
               '2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01',
               '2019-01-01', '2020-01-01'],
              dtype='datetime64[ns]', name='period', freq='AS-JAN')

In [74]:
d.index.get_level_values('period').to_timestamp()

DatetimeIndex(['2007-01-01', '2007-01-01', '2007-01-01', '2008-01-01',
               '2008-01-01', '2008-01-01', '2009-01-01', '2009-01-01',
               '2009-01-01', '2010-01-01', '2010-01-01', '2010-01-01',
               '2011-01-01', '2011-01-01', '2011-01-01', '2012-01-01',
               '2012-01-01', '2012-01-01', '2013-01-01', '2013-01-01',
               '2013-01-01', '2014-01-01', '2014-01-01', '2014-01-01',
               '2015-01-01', '2015-01-01', '2015-01-01', '2016-01-01',
               '2016-01-01', '2016-01-01', '2017-01-01', '2017-01-01',
               '2017-01-01', '2018-01-01', '2018-01-01', '2018-01-01',
               '2019-01-01', '2019-01-01', '2019-01-01', '2020-01-01',
               '2020-01-01', '2020-01-01'],
              dtype='datetime64[ns]', name='period', freq=None)

In [80]:
d.index = d.index.set_levels(d.index.levels[0].to_timestamp(), level=0)

In [82]:
d = d.reset_index()

In [88]:
ht, _, _ = hts.functions.get_hierarchichal_df(
    d.reset_index(),
    level_names=["SIC_Code", "ticker"],
    date_colname="period",
    val_colname="GrossProfit",
    hierarchy=[
        ["SIC_Code"],
        ["ticker"],
    ],
)

In [90]:
heirarchy = {"total": ["7", "2"], "7": ["7_MSFT", "7_ORCL"], "2": ["2_KO"]}

In [96]:
clf = HTSRegressor(model="profet", revision_method="OLS", n_jobs=0)

In [93]:
model = clf.fit(ht, heirarchy)

Fitting models: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.68it/s]


In [94]:
predicted_autoarima = model.predict(steps_ahead=4)

Fitting models:   0%|                                                                                                                                                                                           | 0/6 [00:00<?, ?it/s]c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "
c:\users\andre\appdata\local\programs\python\python38\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In futur

In [95]:
predicted_autoarima

,total,7,2,7_MSFT,7_ORCL,2_KO
2007-01-01,2.202327e+10,6.405520e+09,1.561775e+10,-3.974017e+09,1.959177e+10,6.405520e+09
2008-01-01,8.744524e+10,4.770040e+10,3.974484e+10,3.703965e+10,2.705189e+09,4.770040e+10
2009-01-01,8.905292e+10,4.782022e+10,4.123270e+10,3.478243e+10,6.450266e+09,4.782022e+10
2010-01-01,9.329778e+10,5.049038e+10,4.280740e+10,3.595006e+10,6.857335e+09,5.049038e+10
2011-01-01,1.072439e+11,5.783283e+10,4.941108e+10,3.786327e+10,1.154780e+10,5.783283e+10
2012-01-01,1.155487e+11,6.256118e+10,5.298749e+10,4.184155e+10,1.114595e+10,6.256118e+10
2013-01-01,1.161362e+11,6.328073e+10,5.285548e+10,4.273767e+10,1.011781e+10,6.328073e+10
2014-01-01,1.206291e+11,6.576226e+10,5.486684e+10,4.361537e+10,1.125147e+10,6.576226e+10
2015-01-01,1.189181e+11,6.532057e+10,5.359751e+10,4.518232e+10,8.415186e+09,6.532057e+10
2016-01-01,1.148886e+11,6.305722e+10,5.183133e+10,4.354780e+10,8.283530e+09,6.305722e+10
